In [91]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

# Read recipe inputs
CALCULATED_CARD_DRAW_UPS = dataiku.Dataset("CALCULATED_CARD_DRAW_UPS")
CALCULATED_CARD_DRAW_UPS_df = CALCULATED_CARD_DRAW_UPS.get_dataframe()

CALCULATED_CARD_DRAW_DOWNS = dataiku.Dataset("CALCULATED_CARD_DRAW_DOWNS")
CALCULATED_CARD_DRAW_DOWNS_df = CALCULATED_CARD_DRAW_DOWNS.get_dataframe()

In [92]:
df_down = CALCULATED_CARD_DRAW_DOWNS_df
df_up = CALCULATED_CARD_DRAW_UPS_df

df_down.sort_values(['CUSTOMER'], inplace=True)
df_up.sort_values(['CUSTOMER'], inplace=True)

print(len(df_down), "draw downs")
print(len(df_up), "draw ups")

34557 draw downs
48169 draw ups


In [93]:
df_down.head()

,CUSTOMER,DRAW_DOWN_DATE,MEAN_DD,STD_DD,ACTIVE_CARD_MAX
0,1 AFFORDABLE LAWN MAINTENANCE,2022-02-01 00:00:00+00:00,2.000000,0.000000,3.0
9999,1 BRUDA 3 SISTAS LLC,2021-09-01 00:00:00+00:00,NaN,NaN,1.0
9998,1 G R V LLC,2019-08-01 00:00:00+00:00,0.500000,0.577350,7.0
9997,1 KC LOGISTICS,2021-10-01 00:00:00+00:00,NaN,NaN,1.0
9996,1 PRIORITY ENVIRONMENTAL SERVICES LLC,2021-01-01 00:00:00+00:00,6.333333,0.492366,7.0


In [112]:
import string

class Draw_Down_Customer:

    def __init__(self, name, draw_down_date, mean_dd, std_dd, active_card_max):

        self._common_words = ['1','2','3','4','5','6','7','PIZZA', 'MANAGEMENT', 'THERAPEUTICS', 'USA', 'INC', 'US', 'EQUIPMENT', 'MEDICAL', 'SYSTEMS',
                             'ANIMAL', 'HEALTH', 'LLC', 'CORPORATION', 'BRANDS', 'TIRE', 'RUBBER', 'COUNTRY', 'CORP',
                              'PHARMACY','INC', 'RESTAURANTS', 'CONTAINER', 'AMERICA', 'APPLICATIONS', 'TECHNOLOGY',
                              'INSURANCE', 'FARM','CREDIT', 'SERVICES', 'SERVICE', 'ACCOUNT', 'GENERAL', 'PARTS',
                              'INTL', 'FLAVORS', 'HOLDINGS', 'FOOD','INDUSTRIES', 'LP', 'FLEET', 'MEDICAL', 'PHARMA',
                             'GLOBAL', 'PIPELINE', 'WHEELS', 'BIOSCIENCES', 'SSI', 'SPRINGS', 'NORTH', 'MARINE', 'HOLDING',
                              'TECHNOLOGIES','GROUP', 'PHARMACEUTICAL', 'NA', 'USA', 'COMPANY', 'RAIL', 'PARTNERS', 'BROS',
                              'CO', 'PHARMACEUTICALS', 'ENERGY', 'DISTRIBUTION', 'DENTAL', 'SPECIALTIES', 'OPERATIONS',
                              'COMPANY', 'THE', 'MOUNTAIN', 'TRANS', 'FUEL', 'AMERICAN', 'HOMES', 'GAS',
                             'AFFORDABLE', 'LAWN', 'MAINTENANCE','LOGISTICS','INC','LLC','CO','CORP','LTD','LIMITED',
                              'IN','COMPANY','COMPANIES','VENTURES','TECHNOLOGIES','WORKS','AND','THE','OF','COMPANY',
                              'SYSTEMS','PROPERTY','EXPRESS','SONS','BROTHERS','BUILDING','SERVIC','SERVICE','SERVICES',
                              'CONSTRUCTION','CONSTRUCTIONS','ELECTRIC','ELECTIC','ELECTRICAL','ELECTRONICS','GROUPE',
                              'GROUP','SOLUTIONS','PLUMBING','ENTERPRISES','TRANSPORT','TRANSPORTATION','SYSTEMS',
                              'MANAGEMENT','CONTRACTING','ASSOCIATES','CONSULTING','CONTRAC','CONTRACTORS','CONSTRUCTORS',
                              'SECURITY','INDUSTRIES','EXPRESS','SONS','PROPERTIES','INVESTMENTS','INVESTMENT',
                              'CORPORATION','BUILDERS','ENTERPRISE','STORE','INDUSTRIAL','AUTOMOTIVE','ENGINEERING',
                              'INTERNATIONAL','MEDICAL','MOTORS','STATE','COMMUNICATIONS','COMMUNICATION','DELIVERY',
                              'COMMERCIAL','REFRIGERATION','BUSINESS','HOUSING','DEPARTMENT','TECHNOLOGY','FOODS',
                              'PRODUCTIONS','MANUFACTURING','CONTRACTOR','DISTRIBUTORS','SYSTEM','ENTERTAINMENT',
                              'HOSPITAL','OPERATIONS','EXTERIORS','ASSOCIATED','FOUNDATIONS','LABORATORIES','BLACK',
                              'DECK','ENERGY','BUS','CONCRETE','CONTROL','CONTROLS','FIRE','WHEELS','TEST','KM','WINDOW',
                              'CLEANING','STEEMER','DEZURE','WHEELS','TEST','EQUIPMENT','COURIER','SANITARY',
                              'PAINT','PARTS','ELECTRICA','PRIORITY', 'ENVIRONMENTAL','WAY', 'AUTO', 'FREIGHT']

        self.CUSTOMER = name
        self.DRAW_DOWN_DATE = draw_down_date
        self.ACTIVE_CARD_MAX = active_card_max

        self.MATCHING_CUSTOMERS = []
        self.PERCENT_DIFFERENCE = []
        self.DAYS_DIFFERENCE = []
        self.DRAW_UP_DATE = []

        # remove punctuation
        c_str = name.translate(str.maketrans('', '', string.punctuation))

        f = c_str.split()
        self.WORD_LIST = []
        for w in f:
            if w not in self._common_words:
                self.WORD_LIST.append(w)

    def Match_Draw_Up_Customer(self, name, draw_up_date, mean_du, std_du, active_card_max):

        if (self.CUSTOMER == name):
            # exact match, already captured
            return

        c_str = name.translate(str.maketrans('', '', string.punctuation))

        f = c_str.split()

        check_list = []
        for w in f:
            if (w not in self._common_words) and len(w)>1:
                check_list.append(w)

        percent_diff = round((abs(self.ACTIVE_CARD_MAX - active_card_max) / ((self.ACTIVE_CARD_MAX+active_card_max)/2)),2)

        #date_format = "%Y-%m-%d"
        #d1_date = datetime.strptime(draw_up_date.astype(str), date_format)
        #d2_date = datetime.strptime(self.DRAW_DOWN_DATE.astype(str), date_format)

        delta_between_drop_and_rise = round(abs((draw_up_date-self.DRAW_DOWN_DATE).days)/30.,0)

        for w_to_check in check_list:
            for w in self.WORD_LIST:
                if w_to_check==w:

                    if not name in(self.MATCHING_CUSTOMERS) and(delta_between_drop_and_rise<=4)and(percent_diff<=0.5) :
                        self.MATCHING_CUSTOMERS.append(name)
                        self.PERCENT_DIFFERENCE.append(percent_diff)
                        self.DAYS_DIFFERENCE.append(delta_between_drop_and_rise)
                        self.DRAW_UP_DATE.append(draw_up_date)
                        break;

In [111]:
idx = 0
_customers = []
verbose = True

max_idx = 10

for index, row in df_down.iterrows():

    idx+=1

    customer = row['CUSTOMER']
    draw_down_date = row['DRAW_DOWN_DATE']
    mean_dd = row['MEAN_DD']
    std_dd = row['STD_DD']
    active_card_max = row['ACTIVE_CARD_MAX']

    c = Draw_Down_Customer(customer, draw_down_date, mean_dd, std_dd, active_card_max)

    _customers.append(c)

    if max_idx>0:
        if idx>10:
            break;

idx = 0

_direct_customer = []
_direct_match = []
_direct_draw_up_date = []

_multiple_customer = []
_multiple_matches = []
_multiple_drop_dates = []

for c in _customers:

    for index_up, row_up in df_up.iterrows():

        idx+=1

        customer = row_up['CUSTOMER']
        draw_up_date = row_up['DRAW_UP_DATE']
        mean_du = row_up['MEAN_DU']
        std_du = row_up['STD_DU']
        active_card_max = row_up['ACTIVE_CARD_MAX']

        c.Match_Draw_Up_Customer(customer, draw_up_date, mean_du, std_du, active_card_max)

    if len(c.MATCHING_CUSTOMERS)==1:

        _direct_customer.append(c.CUSTOMER)
        _direct_match.append(c.MATCHING_CUSTOMERS[0])
        _direct_draw_up_date.append(c.DRAW_UP_DATE[0])

        print(c.CUSTOMER, c.MATCHING_CUSTOMERS)

        if verbose:
            print("DIRECT")
            print(c.CUSTOMER, c.WORD_LIST)
            print("Draw Up Date:", c.DRAW_DOWN_DATE)
            print("Cards", c.ACTIVE_CARD_MAX)
            print()
            print(c.MATCHING_CUSTOMERS)
            print(c.PERCENT_DIFFERENCE)
            print(c.DAYS_DIFFERENCE)
            print()
            print("=====")
            print()

    elif len(c.MATCHING_CUSTOMERS)>1:

        _multiple_customer.append(c.CUSTOMER)
        _multiple_matches.append(c.MATCHING_CUSTOMERS)
        _multiple_drop_dates.append(c.DRAW_UP_DATE)

        if verbose:
            print("MULTIPLE")
            print(c.CUSTOMER, c.WORD_LIST)
            print("Draw Up Date:", c.DRAW_DOWN_DATE)
            print("Cards", c.ACTIVE_CARD_MAX)
            print()
            print(c.MATCHING_CUSTOMERS)
            print(c.PERCENT_DIFFERENCE)
            print(c.DAYS_DIFFERENCE)
            print()
            print("=====")
            print()

print(idx)
print()

print(len(_direct_customer), "direct matches")
print(len(_multiple_matches), "multiple matches")

MULTIPLE
1 PRIORITY ENVIRONMENTAL SERVICES LLC ['PRIORITY', 'ENVIRONMENTAL']
Draw Up Date: 2021-01-01 00:00:00+00:00
Cards 7.0

['CORE ENVIRONMENTAL SOLUTIONS', 'JAG ENVIRONMENTAL LLC', 'MISSION EARTHWORK AND ENVIRONMENTAL INC', 'MW ENVIRONMENTAL LLC', 'PECKER HEADS TREE  ENVIRONMENTAL LLC', 'RIO GRANDE ENVIRONMENTAL']
[0.35, 0.15, 0.33, 0.13, 0.35, 0.15]
[4.0, 1.0, 3.0, 2.0, 1.0, 4.0]

=====

MULTIPLE
1 WAY AUTO AND FREIGHT LOGISTICS ['WAY', 'AUTO', 'FREIGHT']
Draw Up Date: 2021-11-01 00:00:00+00:00
Cards 2.0

['A PLUS AUTO SALES', 'A R F AUTO BODY PAINT  SUPPLY INC', 'ABOVE ALL FREIGHT  CO LLC', 'AER AUTO REPAIR LLC', 'AJ AUTO DEALER LLC', 'ALEXANDERS AUTO PARTS INC', 'ALL TYPE AUTO PARTS  REPAIR', 'AMERICAN AUTO GLASS', 'AQUIRECORPS NORWALK AUTO AUCTION', 'ARM FREIGHT CARRIERS', 'AUSTIN  SON AUTO ELECTRIC INC', 'AUSTIN UPULLIT AUTO PARTS INC', 'AUTO CONNECTION OF BOILING SPRINGS LLC', 'AUTO GLASS MIDFLORIDA INC', 'AUTO LOGICS LTD', 'AUTO SAVVY OF SANA FE', 'AUTO WAX  SUPPLY INC', 'B

KeyboardInterrupt: 

In [0]:
df_matches = pd.DataFrame(_direct_customer)
df_matches.columns = ['CUSTOMER']
df_matches["MATCH_CUSTOMER"] = _direct_match
df_matches["DRAW_UP_DATE"] = _direct_draw_up_date
df_matches.head()

In [100]:
df_multiple_matches = pd.DataFrame(_multiple_customer)
df_multiple_matches.columns = ['CUSTOMER']
df_multiple_matches["MATCH_CUSTOMER"] = _multiple_matches
df_multiple_matches["DRAW_UP_DATE"] = _multiple_drop_dates
df_multiple_matches.head()

,CUSTOMER,MATCH_CUSTOMER
0,1 AFFORDABLE LAWN MAINTENANCE,"[1 AFFORDABLE LAWN MAINTENANCE LLC, 5 STAR MAI..."
1,1 G R V LLC,"[B R CONSTRUCTION LLC, B AND R PLUMBING LLC, ..."
2,1 KC LOGISTICS,"[1ST ONE LOGISTICS CORPORATION, 24 7 LOGISTICS..."
3,1 PRIORITY ENVIRONMENTAL SERVICES LLC,"[CORE ENVIRONMENTAL SOLUTIONS, JAG ENVIRONMENT..."
4,1 WAY AUTO AND FREIGHT LOGISTICS,"[1845 LOGISTICS LLC, 230 LOGISTICS LLC, 7FOLDS..."
5,"1 WITH NATURE, LLC","[ANYTHING WITH INK WESTAZ LLC, FLEAUX WITH MO]"
6,10 33 AMBULANCE SERVICE LTD(04,"[1033 AMBULANCE SERVICE LTD, ALLEN COUNTY AMBU..."
7,1005 FINANCIAL GROUP,"[AGZ FINANCIAL SERVICES LLC, BRIGHTER BEGINNIN..."
8,1021 AUTO SALES LLC,"[AR SALES LLC, AUTO DETAILING SOLUTIONS OF ABI..."


In [0]:
MATCHES_1_TO_N_FOR_MANUAL_REVIEW_df = df_multiple_matches
MACTHES_1_TO_N_FOR_MANUAL_REVIEW = dataiku.Dataset("MATCHES_1_TO_N_FOR_MANUAL_REVIEW")
MACTHES_1_TO_N_FOR_MANUAL_REVIEW.write_with_schema(MATCHES_1_TO_N_FOR_MANUAL_REVIEW_df)

MATCHES_1_TO_1_df = df_matches
MATCHES_1_TO_1 = dataiku.Dataset("MATCHES_1_TO_1")
MATCHES_1_TO_1.write_with_schema(MATCHES_1_TO_1_df)